# Tutorial on adding own data as a reaction dependent component
Properties are calculated using the modified Ryzhenko-Bryzgalin (MRB) model, see Gems3-app/Resources/doc/pdf/T-corrections-Reac.pdf

### Import thermohubclient, thermofun, and get the conncetion to the remote database

In [ ]:
import thermohubclient as client
import thermofun as fun
dbc = client.DatabaseClient()

### Get mines19 database to thermofun database

In [ ]:
mines19db = fun.Database(dbc.getDatabase("mines19"))

### Add a reaction as JSON string
{
  "TPMethods":[ ],
  "datasources":"",
  "equation":"",
  "symbol":""
}

### print available methods

In [ ]:
print(fun.availableReactionTPMethods())

### Add LaSO4+_mi = La+3 + SO4-2 reaction from Migdisov et al., 2016 paper, see file LaSO4-reaction.json

In [ ]:
mines19db.setReaction('''
{
  "TPMethods":[
    {
      "dr_ryzhenko_coeffs":{
        "values":[
          3.61,
          1.563,
          -193.7
        ]
      },
      "method":{
        "10":"solute_eos_ryzhenko_gems"
      }
    }
  ],
  "datasources":"Migdisov et al., 2016",
  "equation":"LaSO4+_mi = La+3 + SO4-2",
  "symbol":"LaSO4+_mi"
}
''')

### create a batch object,set units digits for 'logKr', print(fun.availablePropertiesReaction())

In [ ]:
batch = fun.ThermoBatch(mines19db)
batch.setPropertiesUnits(["temperature", "pressure"],["degC","bar"])
batch.setPropertiesDigits(["logKr"], [4])

### set pressure and temperature incremenets

In [ ]:
batch.setPressureIncrement(0,0,0)
batch.setTemperatureIncrement(0,300, 5)

### calculate logK of ["LaSO4+_mi", "La(SO4)+ = La+3 + SO4-2"], new and from mines19

In [ ]:
batch.thermoPropertiesReaction(["LaSO4+_mi", "La(SO4)+ = La+3 + SO4-2"], ["logKr"]).toCSV("res.csv")

### plot resuts 'from functions import plot_properties_vs_temperature'
plot_properties_vs_temperature('res.csv', ["LaSO4+_mi", "La(SO4)+ = La+3 + SO4-2"], 0)

In [ ]:
from common.functions import plot_properties_vs_temperature
plt = plot_properties_vs_temperature('res.csv', ["LaSO4+_mi", "La(SO4)+ = La+3 + SO4-2"], 0)
#plt.ylim(-5, 0)
#plt.legend(loc = "upper left")

### add substance as reaction dependent see LaSO4-reaction_dependent_substance.json

In [ ]:
mines19db.setSubstance('''
        {
            "symbol": "LaSO4+_mi",
            "formula": "LaSO4+",
            "reaction": "LaSO4+_mi"
        }
''')

### create an engine object and calculate properties at 498.15, 0 (Psat) for "La(SO4)+" and "LaSO4+_mi"

In [ ]:
engine = fun.ThermoEngine(mines19db)

In [ ]:
props1 = engine.thermoPropertiesSubstance(498.15, 0, "La(SO4)+")
props2 = engine.thermoPropertiesSubstance(498.15, 0, "LaSO4+_mi")

In [ ]:
props1.gibbs_energy.val

In [ ]:
props2.gibbs_energy.val

### Calculate solvent properties

In [ ]:
res =engine.propertiesSolvent(473, 2000e5, "H2O@")

In [ ]:
res.density.val

In [ ]:
res.densityT.val

In [ ]:
res.density.ddt

### Append to engine engine.appendData("mydata-thermofun.json") and calculate logK of "NdSO4+_mi"

In [ ]:
engine.appendData("myREEdata-thermofun.json")

In [ ]:
props3 = engine.thermoPropertiesReaction(498.15, 0, "NdSO4+_mi")

In [ ]:
props3.log_equilibrium_constant.val